In [64]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from dotenv import load_dotenv
load_dotenv() 
import os
alchemy_key = os.environ["ALCHEMY"]
import keyring

import requests
import time
import sys
import traceback
import json
import pandas as pd
import sqlalchemy
import uuid
import urllib

from web3 import Web3
from ens import ENS

conn_string = 'DRIVER={ODBC Driver 17 for SQL Server};SERVER=dev-web3.database.windows.net;DATABASE=sap;UID=web3_admin;PWD=' + keyring.get_password("sql-dev-web3", "web3_admin") + ";"
url = f'mssql+pyodbc:///?odbc_connect={urllib.parse.quote_plus(conn_string)}'
engine = sqlalchemy.create_engine(url, fast_executemany=True)

## method that calls an API and retries it xx times in case of issues
def api_call(method:str, url:str, headers=None, payload=None, sleeper=2, retries=20):
    retry_counter = 0
    interupt = False

    while True:
        try:
            response = requests.request(method=method, url=url, headers=headers, data=payload)
            if response.status_code == 200:
                break
            else:
                retry_counter += 1
                if retry_counter <= retries:
                    print("retry API call #" + str(retry_counter) + " with: " + url)
                    time.sleep(retry_counter * sleeper)
                else:
                    print("retrying failed more than " + str(retries) + " times - start over")
                    return False
        except KeyboardInterrupt:
            interupt = True
            break        
        except:
            print('request issue, will retry with: ' + url)
            time.sleep(retry_counter * sleeper)
    
    if interupt == True:
        print("Execution ended successfully in api_call")
        sys.exit()
    else:
        return json.loads(response.text)

## iterate over api pagination
def api_load(df:pd.DataFrame, payload_txt:json, headers:str, url:str):
    pageKey = ''
    payload = json.dumps(payload_txt)

    while True:
        try:
            if pageKey != '':
                payload_txt['params'][0]['pageKey'] = pageKey
                payload = json.dumps(payload_txt)

            response_json = api_call(method="POST", url=url, headers=headers, payload=payload)
            if type(response_json) == bool:
                print("Some API load issue with url: " + url)
                return False

            dfTemp = pd.json_normalize(response_json['result'], record_path='transfers', sep='_')
            df = pd.concat([df,dfTemp])

            if 'pageKey' in response_json['result']:
                pageKey = response_json['result']['pageKey']
                time.sleep(0.2)
            else:
                break
            
        except KeyboardInterrupt:
            print("Execution ended successfully")
            sys.exit()
        except requests.exceptions.RequestException as e:
            print("just a Request error, will retry in 2s...")
            time.sleep(2)
            continue
        except Exception as e:
            print("unexpected error")
            print(e)
            print(traceback.format_exc())
            return False
    return df

def upsert_df( df: pd.DataFrame, table_name: str, primary_key_col: str):
    if df.shape[0] > 0:    
        # get datetime columns 
        date_cols = list(df.select_dtypes(include=['datetime64[ns, UTC]']).columns)
        dtype = {col : sqlalchemy.sql.sqltypes.DateTime for col in date_cols}

        table_name_to_transfer = f"temp_{uuid.uuid4().hex[:6]}"

        # replace the placeholder table with the dataframe
        df.to_sql(table_name_to_transfer, engine, if_exists='replace', index=False, dtype=dtype)

        # building the command terms
        cols_list = df.columns.tolist()
        cols_list_query = f'({(", ".join(cols_list))})'
        sr_cols_list = [f'Source.{i}' for i in cols_list]
        sr_cols_list_query = f'({(", ".join(sr_cols_list))})'
        up_cols_list = [f'{i}=Source.{i}' for i in cols_list]
        up_cols_list_query = f'{", ".join(up_cols_list)}'
            
        # fill values that should be interpreted as "NULL" with None
        def fill_null(vals: list) -> list:
            def bad(val):
                if isinstance(val, type(pd.NA)):
                    return True
                # the list of values you want to interpret as 'NULL' should be 
                # tweaked to your needs
                return val in ['NULL', 'nan', '', '', '-', '?']
            return tuple(i if not bad(i) else None for i in vals)

        # create the list of parameter indicators (?, ?, ?, etc...)
        # and the parameters, which are the values to be inserted
        params = [fill_null(row.tolist()) for _, row in df.iterrows()]
        param_slots = '('+', '.join(['?']*len(df.columns))+')'
            
        cmd = f'''
            MERGE {table_name} as Target
            USING {table_name_to_transfer} AS Source
            ON Target.{primary_key_col}=Source.{primary_key_col}
            
            WHEN NOT MATCHED BY Target THEN
            INSERT {cols_list_query} VALUES {sr_cols_list_query}

            WHEN MATCHED THEN
            UPDATE SET {up_cols_list_query};
            '''

        # execute the command to merge tables
        with engine.begin() as conn:
            conn.execute(cmd)

        engine.execute(f'DROP TABLE "{table_name_to_transfer}"')
    return True

def upsert(df: pd.DataFrame, table_name: str, primary_key_col: str):
    if df.shape[0] > 0:
        if df.shape[0] > 10000:
            print("Batch upload necessary")
            total_length = df.shape[0]
            batch_start = 0
            while batch_start < total_length:
                batch_end = batch_start + 10000
                upsert_df(df=df[batch_start:batch_end], table_name=table_name, primary_key_col=primary_key_col)
                print("Batch " + str(batch_end))
                batch_start = batch_end
        else:
            upsert_df(df=df, table_name=table_name, primary_key_col=primary_key_col)
    return df.shape[0]

def lookup_ens():
    w3 = Web3(Web3.HTTPProvider(f'https://eth-mainnet.g.alchemy.com/v2/{alchemy_key}'))
    ns = ENS.fromWeb3(w3)
    exec_string = "SELECT DISTINCT fromAddress FROM toucan_nct WHERE toAddress = '0x0000000000000000000000000000000000000000'"
    addr_df = pd.read_sql_query(exec_string, con=engine)

    ens_df = pd.DataFrame(columns=['address', 'ens'])
    for addr in addr_df['fromAddress']:
        ens_name = ns.name(addr)
        if ens_name is not None:
            print(ens_name)
            new_row = {'address':addr, 'ens':ens_name}
            ens_df = ens_df.append(new_row, ignore_index=True)
    upsert(ens_df, table_name='toucan_ens', primary_key_col='address')

def get_NCT_contracts():
    w3 = Web3(Web3.HTTPProvider(f'https://polygon-mainnet.g.alchemy.com/v2/{alchemy_key}'))
    abi = [{"inputs":[],"stateMutability":"nonpayable","type":"constructor"},{"anonymous":False,"inputs":[{"indexed":False,"internalType":"address","name":"tco2","type":"address"}],"name":"AddFeeExemptedTCO2","type":"event"},{"anonymous":False,"inputs":[{"indexed":False,"internalType":"address","name":"previousAdmin","type":"address"},{"indexed":False,"internalType":"address","name":"newAdmin","type":"address"}],"name":"AdminChanged","type":"event"},{"anonymous":False,"inputs":[{"indexed":True,"internalType":"address","name":"owner","type":"address"},{"indexed":True,"internalType":"address","name":"spender","type":"address"},{"indexed":False,"internalType":"uint256","name":"value","type":"uint256"}],"name":"Approval","type":"event"},{"anonymous":False,"inputs":[{"indexed":False,"internalType":"string","name":"methodology","type":"string"}],"name":"AttributeMethodologyAdded","type":"event"},{"anonymous":False,"inputs":[{"indexed":False,"internalType":"string","name":"methodology","type":"string"}],"name":"AttributeMethodologyRemoved","type":"event"},{"anonymous":False,"inputs":[{"indexed":False,"internalType":"string","name":"region","type":"string"}],"name":"AttributeRegionAdded","type":"event"},{"anonymous":False,"inputs":[{"indexed":False,"internalType":"string","name":"region","type":"string"}],"name":"AttributeRegionRemoved","type":"event"},{"anonymous":False,"inputs":[{"indexed":False,"internalType":"string","name":"standard","type":"string"}],"name":"AttributeStandardAdded","type":"event"},{"anonymous":False,"inputs":[{"indexed":False,"internalType":"string","name":"standard","type":"string"}],"name":"AttributeStandardRemoved","type":"event"},{"anonymous":False,"inputs":[{"indexed":True,"internalType":"address","name":"beacon","type":"address"}],"name":"BeaconUpgraded","type":"event"},{"anonymous":False,"inputs":[{"indexed":False,"internalType":"address","name":"erc20Addr","type":"address"},{"indexed":False,"internalType":"uint256","name":"amount","type":"uint256"}],"name":"Deposited","type":"event"},{"anonymous":False,"inputs":[{"indexed":False,"internalType":"address","name":"erc20addr","type":"address"}],"name":"ExternalAddressRemovedFromWhitelist","type":"event"},{"anonymous":False,"inputs":[{"indexed":False,"internalType":"address","name":"erc20addr","type":"address"}],"name":"ExternalAddressWhitelisted","type":"event"},{"anonymous":False,"inputs":[{"indexed":False,"internalType":"uint8","name":"version","type":"uint8"}],"name":"Initialized","type":"event"},{"anonymous":False,"inputs":[{"indexed":False,"internalType":"address","name":"erc20addr","type":"address"}],"name":"InternalAddressBlacklisted","type":"event"},{"anonymous":False,"inputs":[{"indexed":False,"internalType":"address","name":"erc20addr","type":"address"}],"name":"InternalAddressRemovedFromBlackList","type":"event"},{"anonymous":False,"inputs":[{"indexed":False,"internalType":"address","name":"erc20addr","type":"address"}],"name":"InternalAddressRemovedFromWhitelist","type":"event"},{"anonymous":False,"inputs":[{"indexed":False,"internalType":"address","name":"erc20addr","type":"address"}],"name":"InternalAddressWhitelisted","type":"event"},{"anonymous":False,"inputs":[{"indexed":False,"internalType":"string","name":"mappingName","type":"string"},{"indexed":False,"internalType":"bool","name":"accepted","type":"bool"}],"name":"MappingSwitched","type":"event"},{"anonymous":False,"inputs":[{"indexed":False,"internalType":"uint256","name":"minimumVintageStartTime","type":"uint256"}],"name":"MinimumVintageStartTimeUpdated","type":"event"},{"anonymous":False,"inputs":[{"indexed":True,"internalType":"address","name":"previousOwner","type":"address"},{"indexed":True,"internalType":"address","name":"newOwner","type":"address"}],"name":"OwnershipTransferred","type":"event"},{"anonymous":False,"inputs":[{"indexed":False,"internalType":"address","name":"account","type":"address"}],"name":"Paused","type":"event"},{"anonymous":False,"inputs":[{"indexed":False,"internalType":"address","name":"redeemer","type":"address"},{"indexed":False,"internalType":"uint256","name":"fees","type":"uint256"}],"name":"RedeemFeeBurnt","type":"event"},{"anonymous":False,"inputs":[{"indexed":False,"internalType":"address","name":"redeemer","type":"address"},{"indexed":False,"internalType":"uint256","name":"fees","type":"uint256"}],"name":"RedeemFeePaid","type":"event"},{"anonymous":False,"inputs":[{"indexed":False,"internalType":"address","name":"account","type":"address"},{"indexed":False,"internalType":"address","name":"erc20","type":"address"},{"indexed":False,"internalType":"uint256","name":"amount","type":"uint256"}],"name":"Redeemed","type":"event"},{"anonymous":False,"inputs":[{"indexed":False,"internalType":"address","name":"tco2","type":"address"}],"name":"RemoveFeeExemptedTCO2","type":"event"},{"anonymous":False,"inputs":[{"indexed":True,"internalType":"bytes32","name":"role","type":"bytes32"},{"indexed":True,"internalType":"bytes32","name":"previousAdminRole","type":"bytes32"},{"indexed":True,"internalType":"bytes32","name":"newAdminRole","type":"bytes32"}],"name":"RoleAdminChanged","type":"event"},{"anonymous":False,"inputs":[{"indexed":True,"internalType":"bytes32","name":"role","type":"bytes32"},{"indexed":True,"internalType":"address","name":"account","type":"address"},{"indexed":True,"internalType":"address","name":"sender","type":"address"}],"name":"RoleGranted","type":"event"},{"anonymous":False,"inputs":[{"indexed":True,"internalType":"bytes32","name":"role","type":"bytes32"},{"indexed":True,"internalType":"address","name":"account","type":"address"},{"indexed":True,"internalType":"address","name":"sender","type":"address"}],"name":"RoleRevoked","type":"event"},{"anonymous":False,"inputs":[{"indexed":False,"internalType":"uint256","name":"newCap","type":"uint256"}],"name":"SupplyCapUpdated","type":"event"},{"anonymous":False,"inputs":[{"indexed":False,"internalType":"address[]","name":"tco2s","type":"address[]"}],"name":"TCO2ScoringUpdated","type":"event"},{"anonymous":False,"inputs":[{"indexed":False,"internalType":"address","name":"ContractRegistry","type":"address"}],"name":"ToucanRegistrySet","type":"event"},{"anonymous":False,"inputs":[{"indexed":True,"internalType":"address","name":"from","type":"address"},{"indexed":True,"internalType":"address","name":"to","type":"address"},{"indexed":False,"internalType":"uint256","name":"value","type":"uint256"}],"name":"Transfer","type":"event"},{"anonymous":False,"inputs":[{"indexed":False,"internalType":"address","name":"account","type":"address"}],"name":"Unpaused","type":"event"},{"anonymous":False,"inputs":[{"indexed":True,"internalType":"address","name":"implementation","type":"address"}],"name":"Upgraded","type":"event"},{"inputs":[],"name":"DEFAULT_ADMIN_ROLE","outputs":[{"internalType":"bytes32","name":"","type":"bytes32"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"MANAGER_ROLE","outputs":[{"internalType":"bytes32","name":"","type":"bytes32"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"PAUSER_ROLE","outputs":[{"internalType":"bytes32","name":"","type":"bytes32"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"bool","name":"addToList","type":"bool"},{"internalType":"string[]","name":"_regions","type":"string[]"},{"internalType":"string[]","name":"_standards","type":"string[]"},{"internalType":"string[]","name":"_methodologies","type":"string[]"}],"name":"addAttributes","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"_address","type":"address"}],"name":"addRedeemFeeExemptedAddress","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"_tco2","type":"address"}],"name":"addRedeemFeeExemptedTCO2","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address[]","name":"erc20Addr","type":"address[]"}],"name":"addToExternalWhiteList","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address[]","name":"erc20Addr","type":"address[]"}],"name":"addToInternalBlackList","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address[]","name":"erc20Addr","type":"address[]"}],"name":"addToInternalWhiteList","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"owner","type":"address"},{"internalType":"address","name":"spender","type":"address"}],"name":"allowance","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"spender","type":"address"},{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"approve","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"account","type":"address"}],"name":"balanceOf","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"_account","type":"address"},{"internalType":"uint256","name":"_amount","type":"uint256"}],"name":"bridgeBurn","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"_account","type":"address"},{"internalType":"uint256","name":"_amount","type":"uint256"}],"name":"bridgeMint","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address[]","name":"tco2s","type":"address[]"},{"internalType":"uint256[]","name":"amounts","type":"uint256[]"}],"name":"calculateRedeemFees","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"erc20Addr","type":"address"}],"name":"checkAttributeMatching","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"erc20Addr","type":"address"}],"name":"checkEligible","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"contractRegistry","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"decimals","outputs":[{"internalType":"uint8","name":"","type":"uint8"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"spender","type":"address"},{"internalType":"uint256","name":"subtractedValue","type":"uint256"}],"name":"decreaseAllowance","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"erc20Addr","type":"address"},{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"","type":"address"}],"name":"externalWhiteList","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"feeRedeemBurnAddress","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"feeRedeemBurnPercentageInBase","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"feeRedeemDivider","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"feeRedeemPercentageInBase","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"feeRedeemReceiver","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"getRemaining","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"bytes32","name":"role","type":"bytes32"}],"name":"getRoleAdmin","outputs":[{"internalType":"bytes32","name":"","type":"bytes32"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"getScoredTCO2s","outputs":[{"internalType":"address[]","name":"","type":"address[]"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"bytes32","name":"role","type":"bytes32"},{"internalType":"address","name":"account","type":"address"}],"name":"grantRole","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"bytes32","name":"role","type":"bytes32"},{"internalType":"address","name":"account","type":"address"}],"name":"hasRole","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"spender","type":"address"},{"internalType":"uint256","name":"addedValue","type":"uint256"}],"name":"increaseAllowance","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"initialize","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"","type":"address"}],"name":"internalBlackList","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"","type":"address"}],"name":"internalWhiteList","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"string","name":"","type":"string"}],"name":"methodologies","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"methodologiesIsAcceptedMapping","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"minimumVintageStartTime","outputs":[{"internalType":"uint64","name":"","type":"uint64"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"name","outputs":[{"internalType":"string","name":"","type":"string"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"owner","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"pause","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"paused","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"proxiableUUID","outputs":[{"internalType":"bytes32","name":"","type":"bytes32"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"tco2","type":"address"},{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"redeemAndBurn","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"redeemAuto","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"redeemAuto2","outputs":[{"internalType":"address[]","name":"tco2s","type":"address[]"},{"internalType":"uint256[]","name":"amounts","type":"uint256[]"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"","type":"address"}],"name":"redeemFeeExemptedAddresses","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"","type":"address"}],"name":"redeemFeeExemptedTCO2s","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address[]","name":"tco2s","type":"address[]"},{"internalType":"uint256[]","name":"amounts","type":"uint256[]"}],"name":"redeemMany","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"string","name":"","type":"string"}],"name":"regions","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"regionsIsAcceptedMapping","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address[]","name":"erc20Addr","type":"address[]"}],"name":"removeFromExternalWhiteList","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address[]","name":"erc20Addr","type":"address[]"}],"name":"removeFromInternalBlackList","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address[]","name":"erc20Addr","type":"address[]"}],"name":"removeFromInternalWhiteList","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"_address","type":"address"}],"name":"removeRedeemFeeExemptedAddress","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"_tco2","type":"address"}],"name":"removeRedeemFeeExemptedTCO2","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"renounceOwnership","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"bytes32","name":"role","type":"bytes32"},{"internalType":"address","name":"account","type":"address"}],"name":"renounceRole","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"bytes32","name":"role","type":"bytes32"},{"internalType":"address","name":"account","type":"address"}],"name":"revokeRole","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"router","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint256","name":"","type":"uint256"}],"name":"scoredTCO2s","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"seedMode","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"_feeRedeemBurnAddress","type":"address"}],"name":"setFeeRedeemBurnAddress","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint256","name":"_feeRedeemBurnPercentageInBase","type":"uint256"}],"name":"setFeeRedeemBurnPercentage","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint256","name":"_feeRedeemPercentageInBase","type":"uint256"}],"name":"setFeeRedeemPercentage","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"_feeRedeemReceiver","type":"address"}],"name":"setFeeRedeemReceiver","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint64","name":"_minimumVintageStartTime","type":"uint64"}],"name":"setMinimumVintageStartTime","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"_router","type":"address"}],"name":"setRouter","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint256","name":"newCap","type":"uint256"}],"name":"setSupplyCap","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address[]","name":"tco2s","type":"address[]"}],"name":"setTCO2Scoring","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"_address","type":"address"}],"name":"setToucanContractRegistry","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"string","name":"","type":"string"}],"name":"standards","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"standardsIsAcceptedMapping","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"supplyCap","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"bytes4","name":"interfaceId","type":"bytes4"}],"name":"supportsInterface","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"string","name":"_mappingName","type":"string"},{"internalType":"bool","name":"accepted","type":"bool"}],"name":"switchMapping","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"symbol","outputs":[{"internalType":"string","name":"","type":"string"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"","type":"address"}],"name":"tokenBalances","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"totalSupply","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"recipient","type":"address"},{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"transfer","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"sender","type":"address"},{"internalType":"address","name":"recipient","type":"address"},{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"transferFrom","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"newOwner","type":"address"}],"name":"transferOwnership","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"unpause","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"newImplementation","type":"address"}],"name":"upgradeTo","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"newImplementation","type":"address"},{"internalType":"bytes","name":"data","type":"bytes"}],"name":"upgradeToAndCall","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[],"name":"version","outputs":[{"internalType":"string","name":"","type":"string"}],"stateMutability":"pure","type":"function"}]
    address = '0xD838290e877E0188a4A44700463419ED96c16107'
    myContract = w3.eth.contract(address=address, abi=abi)
    return myContract.caller.getScoredTCO2s()

class load:
    def __init__(self, load_type:str):
        self.load_type = load_type

        if self.load_type == 'NCT':
            self.table_name = 'toucan_nct'
            self.primary_key_col = 'uniqueId'

            self.df_columns = ['blockNum', 'uniqueId', 'hash', 'from', 'to', 'value', 'erc721TokenId', 'erc1155Metadata', 'tokenId', 'asset', 'category', 'rawContract_value', 'rawContract_address', 'rawContract_decimal', 'metadata_blockTimestamp']
            self.method = "POST"
            self.url = f"https://polygon-mainnet.g.alchemy.com/v2/{alchemy_key}"
            
            contractAddresses = get_NCT_contracts()
            self.payload_txt = {
                "id": 1,
                "jsonrpc": "2.0",
                "method": "alchemy_getAssetTransfers",
                "params": [
                    {
                        "fromBlock": "0x0",
                        "toBlock": "latest",
                        "contractAddresses": contractAddresses,
                        "category": [
                                "erc20"
                        ],
                        "withMetadata": True,
                        "excludeZeroValue": True,
                        "maxCount": "0x3e8"
                    }
                ]
            }
            self.headers = {'Content-Type': 'text/plain'}

    def create_df(self):
        df = pd.DataFrame(columns = self.df_columns)
        return df

 
    def transform(self, df:pd.DataFrame):
        if self.load_type == 'NCT':
            df.drop(['erc721TokenId', 'erc1155Metadata', 'tokenId','rawContract_value','rawContract_decimal'], axis=1, inplace=True)
            int_columns = ["value"]    
            df[int_columns] = df[int_columns].apply(pd.to_numeric)
            date_columns = ["metadata_blockTimestamp"]
            df[date_columns] = df[date_columns].apply(pd.to_datetime)
            df.rename(columns= {'rawContract_address':'contractAddress', 'from':'fromAddress', 'to':'toAddress', 'metadata_blockTimestamp':'timestamp'}, inplace=True)
        return df

    def run(self):
        df = self.create_df()
        df = api_load(df=df, payload_txt=self.payload_txt, headers=self.headers, url=self.url)

        if df.shape[0] == 0:
            return None
        else:
            df = self.transform(df)
            loaded = upsert(df, table_name=self.table_name , primary_key_col=self.primary_key_col)
            return loaded


In [65]:
loader = load('NCT')
loader.run()

1607

In [ ]:
lookup_ens()